In [1]:
#Dev

import pandas as pd
import numpy as np
# this part needed for mac
# import pymysql
# pymysql.install_as_MySQLdb()
############################
import seaborn as sns
from mysql.connector import MySQLConnection, Error
from XML2MySQL import connect
from MySQL2PandasDF import MySQL2PandasDF
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
% matplotlib inline

# Init Session Object

In [2]:
session = MySQL2PandasDF(host = 'localhost', db = 'stackexchange_travel', user = 'root', password = password)

Connection to MySQL database stackexchange_travel successful.


## db2df method parses the entirety of a database to dict of pandas dfs

In [3]:
session.db2df()

C:\Users\jeffe\Anaconda3\lib\site-packages\sqlalchemy\dialects\mysql\base.py:1546: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  cursor.execute('SELECT @@tx_isolation')


Underlying structure is a dictionary with the following format:

{Table Name: df}

In [4]:
# tables in our dictionary

session.source.keys()

dict_keys(['badges', 'comments', 'posthistory', 'postlinks', 'posts', 'tags', 'users', 'votes'])

## Head of every table in database

In [ ]:
session.source['badges'].head()

In [ ]:
session.source['comments'].head()

In [ ]:
session.source['posthistory'].head()

In [ ]:
session.source['postlinks'].head()

In [ ]:
session.source['posts']['Body']

In [ ]:
session.source['tags'].head()

In [ ]:
session.source['users'].head()

In [12]:
session.source['votes'].head()

## Parse columns with html present

In [13]:
session.parse_html('posts','Body')

## Create new search space feature

In [14]:
def create_search_space():
    """
    This function returns a histogram of post activity for posts tagged with any two tags
    
    params:     tags_search_term1: str: first tag search term to find posts with
                tags_search_term2: str: second tag search term to find posts with
                
    return:     matplotlib.pyplot.hist of number of post reponses for given topic
    """
    # Extract html from body 
    session.source['posts']['Tags_tags_removed'] = session.source['posts']['Tags'].str.findall(r"<(:?.+)>").str.join('').str.replace('>','').str.replace('<',' ')

    # Create search space to find search terms within
    session.source['posts']['Search_Space'] = session.source['posts']['Tags_tags_removed'].fillna('').map(str) + ' ' + session.source['posts']['Title'].fillna('').values + ' ' + session.source['posts']['Body'].fillna('').values

create_search_space()
session.source['posts']['Search_Space']

0        caribbean cruising vacations What are some Car...
1        guides extreme-tourism amazon-river amazon-jun...
2          One way would be to go through an Adventure ...
3        loyalty-programs routes ewr singapore-airlines...
4        romania transportation What is the easiest tra...
5        extreme-tourism antarctica How can I visit Ant...
6        public-transport airport-transfer taxis sea Be...
7        sightseeing public-transport transportation ar...
8        safety international-travel money exchange Whe...
9        russia visas china mongolia trans-siberian Wha...
10         There are a few ways to visit Antarctica. Re...
11       online-resources transportation peru south-ame...
12       us-citizens travel-agents cuba Is it advisable...
13       sightseeing hungary budapest What are some of ...
14       europe online-resources planning guides trains...
15         I think the current usual solution is to get...
16         What I now do and it has worked really well .

# Remove stopwords

In [15]:
from nltk.corpus import stopwords
import string
stop = stopwords.words('english')

def remove_stop_words(text):
    '''
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Return the cleaned text as a list of words
    '''
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    return ' '.join([word for word in nopunc.split() if word.lower() not in stop])

In [16]:
session.source['posts']['Search_Space_stopwords_removed'] = session.source['posts']['Search_Space'].apply(lambda x: remove_stop_words(x))

## Make lemmanade out of the lemmas

In [17]:
def lemma(text):
    '''
    Takes in a string of text, then performs the following:
    1. Reduce all words down to lemma (root word)
    2. Return lemmatized words
    :params: text: str: The string to be made into lemmanade
    :return: text: str: Lemmanade version of input text
    '''
    from nltk.stem.wordnet import WordNetLemmatizer
    lemma = WordNetLemmatizer()
    normalized = " ".join(lemma.lemmatize(word,'v') for word in text.split())
    x = normalized.split()
    y = [s for s in x if len(s) > 2]
    return y

In [18]:
session.source['posts']['Search_Space_stopwords_removed_lemma'] = session.source['posts']['Search_Space_stopwords_removed'].apply(lambda x: lemma(x))

## Build Dictionary to Feed to LDA Model

In [19]:
from gensim import corpora
list_of_post_contents = list()
for post_contents in session.source['posts']['Search_Space_stopwords_removed_lemma']:
    list_of_post_contents.append(post_contents)

dictionary = corpora.Dictionary(list_of_post_contents)

# Filter terms which occurs in less than 4 articles & more than 40% of the articles 
dictionary.filter_extremes(no_below=4, no_above=0.4)

# List of few words which are removed from dictionary as they are content neutral
stoplist = set('also use make people know many call include part find become like mean often different \
               usually take wikt come give well get since type list say change see refer actually iii \
               aisne kinds pas ask would way something need things want every str'.split())
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in list_of_post_contents]

C:\Users\jeffe\Anaconda3\lib\site-packages\gensim-3.2.0-py3.6-win-amd64.egg\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Build LDA Model

In [20]:
from gensim.models.ldamodel import LdaModel as Lda
# Creating the object for LDA model using gensim library & Training LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=20, id2word = dictionary, passes=50, iterations=500)

# Print all the 20 topics
for i,topic in enumerate(ldamodel.print_topics(num_topics=20, num_words=10)):
    words = topic[1].split("+")
    print(words,"\n")

['0.022*"luggage" ', ' 0.019*"bag" ', ' 0.018*"carry" ', ' 0.016*"bring" ', ' 0.015*"allow" ', ' 0.014*"water" ', ' 0.012*"license" ', ' 0.011*"items" ', ' 0.011*"check" ', ' 0.007*"security"'] 

['0.059*"days" ', ' 0.034*"day" ', ' 0.034*"time" ', ' 0.027*"trip" ', ' 0.024*"stay" ', ' 0.021*"months" ', ' 0.020*"week" ', ' 0.020*"year" ', ' 0.017*"month" ', ' 0.015*"back"'] 

['0.046*"name" ', ' 0.038*"travel" ', ' 0.026*"insurance" ', ' 0.025*"English" ', ' 0.023*"Japan" ', ' 0.015*"speak" ', ' 0.015*"countries" ', ' 0.013*"country" ', ' 0.012*"children" ', ' 0.011*"age"'] 

['0.066*"train" ', ' 0.034*"bus" ', ' 0.025*"station" ', ' 0.012*"time" ', ' 0.010*"stop" ', ' 0.010*"transport" ', ' 0.010*"travel" ', ' 0.009*"day" ', ' 0.009*"line" ', ' 0.008*"service"'] 

['0.026*"South" ', ' 0.020*"New" ', ' 0.013*"Zealand" ', ' 0.011*"Africa" ', ' 0.011*"safe" ', ' 0.011*"Member" ', ' 0.011*"State" ', ' 0.011*"Australia" ', ' 0.010*"countries" ', ' 0.010*"travel"'] 

['0.084*"transit" ', ' 

## Visualizing intertopic relationships

In [21]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)

C:\Users\jeffe\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
12     14.694494        1       1 -0.223506  0.016329
19     12.263520        1       2 -0.198070 -0.081613
18     11.619709        1       3 -0.060997 -0.046417
3       6.000531        1       4 -0.158794  0.141888
13      5.094674        1       5 -0.120051 -0.104446
17      4.724189        1       6 -0.134625  0.163054
16      4.420006        1       7 -0.018938  0.099937
14      4.379121        1       8 -0.111150 -0.079256
9       4.310069        1       9  0.063325 -0.241996
0       4.299695        1      10 -0.000726 -0.126648
8       3.866495        1      11 -0.095879 -0.156782
11      3.759371        1      12 -0.006842  0.149502
10      3.474164        1      13 -0.000231  0.048534
1       3.272723        1      14 -0.055533  0.133549
2       2.771530        1      15  0.103400 -0.107506
15      2.671790        1      16  0.125055 -0.003676
5       2.569117        1      17  0.165588  0.243142
4       2.277147        1      18  0.198445 -0.029191
7       2.200346        1      19  0.256161 -0.027215
6       1.331309        1      20  0.273365  0.008813, topic_info=      Category          Freq         Term         Total  loglift  logprob
term                                                                     
250    Default  52770.000000         visa  52770.000000  30.0000  30.0000
244    Default  24762.000000       ticket  24762.000000  29.0000  29.0000
88     Default  26609.000000       flight  26609.000000  28.0000  28.0000
79     Default  17151.000000        train  17151.000000  27.0000  27.0000
186    Default  13110.000000         card  13110.000000  26.0000  26.0000
1717   Default  18552.000000     passport  18552.000000  25.0000  25.0000
57     Default  28040.000000         time  28040.000000  24.0000  24.0000
169    Default  12855.000000         days  12855.000000  23.0000  23.0000
2952   Default  10395.000000      transit  10395.000000  22.0000  22.0000
134    Default  12997.000000      airport  12997.000000  21.0000  21.0000
226    Default  12845.000000         book  12845.000000  20.0000  20.0000
23     Default  31300.000000       travel  31300.000000  19.0000  19.0000
607    Default   8189.000000      airline   8189.000000  18.0000  18.0000
140    Default   9015.000000          bus   9015.000000  17.0000  17.0000
721    Default  13370.000000         stay  13370.000000  16.0000  16.0000
833    Default   6250.000000       Canada   6250.000000  15.0000  15.0000
189    Default  14126.000000        check  14126.000000  14.0000  14.0000
688    Default   7344.000000         name   7344.000000  13.0000  13.0000
3276   Default   8359.000000  application   8359.000000  12.0000  12.0000
434    Default  10922.000000          pay  10922.000000  11.0000  11.0000
453    Default   9858.000000          day   9858.000000  10.0000  10.0000
3257   Default   9845.000000     Schengen   9845.000000   9.0000   9.0000
344    Default   4875.000000      Germany   4875.000000   8.0000   8.0000
363    Default   8653.000000          buy   8653.000000   7.0000   7.0000
608    Default   5923.000000     airlines   5923.000000   6.0000   6.0000
61     Default  11497.000000         trip  11497.000000   5.0000   5.0000
1402   Default   6562.000000      station   6562.000000   4.0000   4.0000
1065   Default   7602.000000      luggage   7602.000000   3.0000   3.0000
113    Default  13896.000000        visit  13896.000000   2.0000   2.0000
191    Default  13937.000000      country  13937.000000   1.0000   1.0000
...        ...           ...          ...           ...      ...      ...
2085   Topic20    997.270962         Iran    998.237434   4.3180  -4.0636
1870   Topic20    877.861866     Pakistan    878.828338   4.3179  -4.1911
3213   Topic20    845.942230     Istanbul    846.908702   4.3179  -4.2282
4933   Topic20    731.900159      Austria    732.866632   4.3177  -4.3730
3577   Topic20    708.845533   